In [1]:

// @hidden_cell
// This function is used to setup the access of Spark to your Object Storage. The definition contains your credentials.
// You might want to remove those credentials before you share your notebook.
def setHadoopConfig64e9c4bd3e4148978db0a312dfcc0a93(name: String) = {
    // This function sets the Hadoop configuration so it is possible to
    // access data from Bluemix Object Storage using Spark

    val prefix = "fs.swift.service." + name
    sc.hadoopConfiguration.set(prefix + ".auth.url", "https://identity.open.softlayer.com" + "/v3/auth/tokens")
    sc.hadoopConfiguration.set(prefix + ".auth.endpoint.prefix","endpoints")
    sc.hadoopConfiguration.set(prefix + ".tenant", "419cd8dece644c82af5a615b62af38e1")
    sc.hadoopConfiguration.set(prefix + ".username", "20ae3d743b904885bcebe7b72c6faeb3")
    sc.hadoopConfiguration.set(prefix + ".password", "s9b#uo2T(T_NoZe(")
    sc.hadoopConfiguration.setInt(prefix + ".http.port", 8080)
    sc.hadoopConfiguration.set(prefix + ".region", "dallas")
    sc.hadoopConfiguration.setBoolean(prefix + ".public", false)
}

// you can choose any name
val name = "keystone"
setHadoopConfig64e9c4bd3e4148978db0a312dfcc0a93(name)

val sqlContext = new org.apache.spark.sql.SQLContext(sc)

val dfData1 = sqlContext.
    read.format("com.databricks.spark.csv").
    option("header", "true").
    option("inferSchema", "true").
    load("swift://BankChurnAnalysis." + name + "/CUST_SUM_CHINA.csv")
dfData1.show(5)


+----------+---+---+---------+----------+------+--------+-----+---------+------------+-------+----------+---------+---------+-----+-----+--------------------+-------------+-----+--------------------+---------+-----------+
|   CUST_ID|SEX|AGE|EDUCATION|INVESTMENT|INCOME|ACTIVITY|CHURN| YRLY_AMT|AVG_DAILY_TX|YRLY_TX|AVG_TX_AMT|NEGTWEETS| PROVINCE|NAMEA|NAMEB|               NAMEC|        NAMED|STATE|     EDUCATION_GROUP|TwitterID|CHURN_LABEL|
+----------+---+---+---------+----------+------+--------+-----+---------+------------+-------+----------+---------+---------+-----+-----+--------------------+-------------+-----+--------------------+---------+-----------+
|1009520370|  F| 63|        4|         0| 13035|       3|    0|6339.0703|    0.663014|    242| 26.194504|        2|Chongqing|    渝|  重庆市|Chongqing Municip...|Chóngqìng Shì|   MD|    Associate degree|      0.0|      FALSE|
|1009520380|  F| 59|        1|         0| 18267|       0|    0|7957.1406|    0.654795|    239| 33.293503|       

In [2]:
dfData1.printSchema

root
 |-- CUST_ID: integer (nullable = true)
 |-- SEX: string (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- EDUCATION: integer (nullable = true)
 |-- INVESTMENT: integer (nullable = true)
 |-- INCOME: integer (nullable = true)
 |-- ACTIVITY: integer (nullable = true)
 |-- CHURN: integer (nullable = true)
 |-- YRLY_AMT: double (nullable = true)
 |-- AVG_DAILY_TX: double (nullable = true)
 |-- YRLY_TX: integer (nullable = true)
 |-- AVG_TX_AMT: double (nullable = true)
 |-- NEGTWEETS: integer (nullable = true)
 |-- PROVINCE: string (nullable = true)
 |-- NAMEA: string (nullable = true)
 |-- NAMEB: string (nullable = true)
 |-- NAMEC: string (nullable = true)
 |-- NAMED: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- EDUCATION_GROUP: string (nullable = true)
 |-- TwitterID: double (nullable = true)
 |-- CHURN_LABEL: string (nullable = true)



In [3]:
%Addjar --magic https://brunelvis.org/jar/spark-kernel-brunel-all-2.3.jar -f

Starting download from https://brunelvis.org/jar/spark-kernel-brunel-all-2.3.jar


In [4]:
%%brunel data('dfData1') map("china") key(Province) label(NAMEA) color(INCOME) mean(INCOME) 

In [13]:
%%brunel data('dfData1') 
         x(AGE) y(#count:linear) color(CHURN_LABEL) bin(AGE) interaction(select) stack bar tooltip(#all) filter(CHURN_LABEL) legends(none) |
         x(AVG_DAILY_TX) y(#count:linear) color(CHURN_LABEL) opacity(#selection) bin(AVG_DAILY_TX) stack bar tooltip(#all) axes(x:10:'AVG DAILY TX', y)|
         x(AVG_TX_AMT) y(#count:linear) color(CHURN_LABEL) opacity(#selection) bin(AVG_TX_AMT) stack bar tooltip(#all)  axes(x,y) legends(none) |
        x(INCOME) y(#count:linear) color(CHURN_LABEL) opacity(#selection) bin(INCOME) stack bar tooltip(#all) tooltip(#all) axes(x,y) legends(none)
:: width=800, height=600

In [14]:
%%brunel data('dfData1')
         x(SEX) y(#count:linear) color(CHURN_LABEL) stack bar tooltip(#all) sort(SEX) interaction(select) filter(CHURN_LABEL) axes(x:'GENDER', y) legends(none) |
         x(ACTIVITY) y(#count:linear) color(CHURN_LABEL) stack bar tooltip(#all) sort(ACTIVITY) opacity(#selection)  | 
         x(EDUCATION_GROUP) y(#count:linear) color(CHURN_LABEL) stack bar tooltip(#all) sort(#count) opacity(#selection)  axes(x:'',y) legends(none)
:: width=900, height=600

In [5]:
%%brunel data('dfData1') map("china") key(PROVINCE) x(PROVINCE) color(CHURN) mean(CHURN) label(NAMEA) tooltip(#all) :: width=800, height=500

In [6]:


//import libraries
import org.apache.spark.ml.feature.{StringIndexer,VectorAssembler}
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.tuning.{CrossValidator,ParamGridBuilder}



In [7]:
val Array(training, test) = dfData1.randomSplit(Array(0.8, 0.2), seed = 11L)
println("The number of training data is ",training.count())
println("The number of test data is ",test.count())

(The number of training data is ,4838)
(The number of test data is ,1163)


In [8]:
//Feature definition
val genderIndexer = new StringIndexer().setInputCol("SEX").setOutputCol("gender_code")
//val NAMEAIndexer = new StringIndexer().setInputCol("NAMEA").setOutputCol("NAMEA")
//val NAMEBIndexer = new StringIndexer().setInputCol("NAMEA").setOutputCol("NAMEA")
//val NAMECIndexer = new StringIndexer().setInputCol("NAMEA").setOutputCol("NAMEA")
//val NAMEDIndexer = new StringIndexer().setInputCol("NAMEA").setOutputCol("NAMEA")
//val provinceIndexer = new StringIndexer().setInputCol("PROVINCE").setOutputCol("NAMEA")
val stateIndexer = new StringIndexer().setInputCol("STATE").setOutputCol("state_code")
val labelIndexer = new StringIndexer().setInputCol("CHURN").setOutputCol("label")

val featuresAssembler = new VectorAssembler().setInputCols(Array("AGE", 
                                                         "ACTIVITY", 
                                                         "EDUCATION", 
                                                         "NEGTWEETS" ,
                                                         "INCOME",
                                                         "gender_code",
                                                         "state_code")).setOutputCol("features")

In [9]:
//Select model automatically in candidate algorithm - Logistic Regression, SVM or Decision Tree?
val lr = new LogisticRegression().setRegParam(0.01).setLabelCol("label").setFeaturesCol("features")
val pipeline =new Pipeline().setStages(Array(labelIndexer, genderIndexer, stateIndexer, featuresAssembler,lr))
val auc_eval = new BinaryClassificationEvaluator()
val grid = new ParamGridBuilder().addGrid(lr.regParam, Array(1e-3, 1e-2)).addGrid(lr.elasticNetParam,Array(0.25, 0.0)).build()
val cross_val = new CrossValidator().setEstimator(pipeline).setEvaluator(auc_eval).setEstimatorParamMaps(grid).setNumFolds(3)

In [10]:
val pipeline_model = cross_val.fit(training)


In [12]:
val trainResult=pipeline_model.transform(test)

In [13]:
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
val testingResultScores = trainResult.select("prediction","label").rdd.map(r => (r(0).asInstanceOf[Double], r(1) .asInstanceOf[Double]))
val bc = new BinaryClassificationMetrics(testingResultScores)
val roc = bc.roc

In [15]:
val rocDF = sqlContext.createDataFrame(roc).
                    withColumnRenamed("_1", "FPR").
                    withColumnRenamed("_2", "TPR")
rocDF.show()

In [18]:
rocDF.show()

+--------------------+------------------+
|                 FPR|               TPR|
+--------------------+------------------+
|                 0.0|               0.0|
|0.004405286343612335|0.9490196078431372|
|                 1.0|               1.0|
|                 1.0|               1.0|
+--------------------+------------------+



In [ ]:
%AddJar -magic https://brunelvis.org/jar/spark-kernel-brunel-all-2.3.jar -f

In [19]:
%%brunel data('rocDF') x(FPR) y(TPR) line tooltip(#all) axes(x:'False Positive Rate':grid, y:'True Positive Rate':grid)